In [ ]:
# задача 3
N = int(input())

commands = {
    "zero": 0,
    "one" : 1,
    "two": 2,
    "three": 3,
    "four": 4,
    "five": 5,
    "six": 6,
    "seven": 7,
    "eight": 8,
    "nine": 9,
    "ten": 10,
    "eleven": 11,
    "twelve": 12,
    "thirteen": 13,
    "fourteen": 14,
    "fifteen": 15,
    "sixteen": 16,
    "seventeen": 17,
    "eighteen": 18,
    "nineteen": 19,
    "twenty": 20,
    "thirty": 30,
    "forty": 40,
    "fifty": 50,
    "sixty": 60,
    "seventy": 70,
    "eighty": 80,
    "ninety": 90
}

reverse_commands = {
    0 : "zero",
    1 : "one",
    2 : "two",
    3 : "three",
    4 : "four",
    5 : "five",
    6 : "six",
    7 : "seven",
    8 : "eight",
    9 : "nine",
    10 : "ten",
    11 : "eleven",
    12 : "twelve",
    13 : "thirteen",
    14 : "fourteen",
    15 : "fifteen",
    16 : "sixteen",
    17 : "seventeen",
    18 : "eighteen",
    19 : "nineteen",
    20 : "twenty",
    30 : "thirty",
    40 : "forty",
    50 : "fifty",
    60 : "sixty",
    70 : "seventy",
    80 : "eighty",
    90 : "ninety"
}

for i in range(N):
    line = input()
    list = line.split(' ')
    
    operator = ""
    n1 = n2 = 0
    result = 0

    for i in list:
        if i == "minus" or i == "plus": operator = i
        else:
            if operator == "": n1 += commands[i]
            else: n2 += commands[i]
    
    if operator == "minus": result = n1 - n2
    else: result = n1 + n2
    
    if result < 20 or result%10 == 0: print(reverse_commands[result])
    else: print(reverse_commands[result - result%10], " ", reverse_commands[result%10])

In [2]:
# задача 4
# Для решения задачи участникам предоставляется информация о транзакциях клиентов банка. Объемом около 27000000 миллионов записей. Каждая запись описывает одну банковскую транзакцию. Для каждого из ≈ 20000 тестовых id, участникам необходимо с помощью обученной модели предсказать, в какую из возрастных групп попадает Клиент.

# Подготовлено два набора данных:

# Обучающий transactions_train.csv, в котором для каждой транзакции известна дата, сумма, тип и id клиента;
# Тестовый transactions_test.csv, содержащий те же поля:
# сlient_id – уникальный номер клиента;
# trans_date – дата транзакции (представляет из себя просто номер дня в хронологическом порядке, начиная от заданной даты);
# small_group – группа транзакций, характеризующих тип транзакции (например, продуктовые магазины, одежда, заправки, детские товары и т.п.);
# amount_rur – сумма транзакции (для анонимизации данные суммы были трансформированы без потери структуры).
# На базе данных файлов можно строить различные признаки, которые характеризуют возрастные группы.

# Целевая переменная для обучающего датасета находится в файле train_target.csv.

# В нем содержится информация о Клиенте и метка возрастной группы, к которой он относится:

# client_id – уникальный номер Клиента (соответствует client_id и файла transactions_train.csv);
# bins – метка возраста. В файлe test.csv вам надо предсказать для указанных client_id соответствующую метку группы возраста.
# Участникам также предоставлен информационный файл small_group_description.csv, который содержит расшифровку типов транзакций.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
transactions_train = pd.read_csv('/content/drive/MyDrive/Data Science/11 КЛАСС/transactions_train.csv')
train_target = pd.read_csv('/content/drive/MyDrive/Data Science/11 КЛАСС/train_target.csv')

In [58]:
transactions_test = pd.read_csv('/content/drive/MyDrive/Data Science/11 КЛАСС/transactions_test.csv')
test_target = pd.read_csv('/content/drive/MyDrive/Data Science/11 КЛАСС/test.csv')

In [7]:
transactions_train.head()

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017
2,33172,8,11,13.887
3,33172,9,11,15.983
4,33172,10,11,21.341


In [8]:
train_target.head()

,client_id,bins
0,24662,2
1,1046,0
2,34089,2
3,34848,1
4,47076,3


In [9]:
transactions_test.head()

,client_id,trans_date,small_group,amount_rur
0,46445,3,0,19.555
1,46445,3,1,27.774
2,46445,4,0,18.114
3,46445,4,1,22.183
4,46445,5,2,45.795


In [59]:
test_target.head()

,client_id
0,28571
1,27046
2,13240
3,19974
4,10505


In [46]:
info_transac_train = transactions_train.groupby('client_id')['amount_rur'].agg(['min', 'max', 'sum', 'mean'])
info_transac_train

,min,max,sum,mean
client_id,,,,
4,0.043,1341.802,28404.121,39.450168
6,0.045,315.781,15720.739,21.535259
7,0.043,4505.971,53630.036,69.379089
10,0.045,654.893,34419.365,48.752642
11,0.388,2105.058,26789.404,32.991877
...,...,...,...,...
49993,0.211,1315.470,24247.544,26.911814
49995,0.690,1243.601,27951.156,28.845362
49996,0.458,1657.546,80900.345,71.089934


In [53]:
amount_transactions_train = transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
counts_train = amount_transactions_train.reset_index().pivot(index='client_id', columns='small_group',values='amount_rur')
counts_train = counts_train.fillna(0)
counts_train.columns = ['small_group_'+str(i) for i in counts_train.columns]
counts_train

,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,small_group_10,small_group_11,small_group_12,small_group_13,small_group_14,small_group_15,small_group_16,small_group_17,small_group_18,small_group_19,small_group_20,small_group_21,small_group_22,small_group_23,small_group_24,small_group_25,small_group_26,small_group_27,small_group_28,small_group_29,small_group_30,small_group_31,small_group_32,small_group_33,small_group_34,small_group_35,small_group_36,small_group_37,small_group_38,small_group_39,...,small_group_162,small_group_163,small_group_164,small_group_165,small_group_166,small_group_167,small_group_168,small_group_169,small_group_170,small_group_171,small_group_172,small_group_173,small_group_174,small_group_175,small_group_176,small_group_177,small_group_178,small_group_179,small_group_180,small_group_181,small_group_182,small_group_183,small_group_184,small_group_185,small_group_186,small_group_187,small_group_188,small_group_189,small_group_190,small_group_191,small_group_192,small_group_193,small_group_195,small_group_196,small_group_197,small_group_198,small_group_199,small_group_200,small_group_202,small_group_203
client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4,0.0,447.0,1.0,44.0,93.0,0.0,0.0,0.0,1.0,13.0,1.0,19.0,0.0,0.0,0.0,37.0,0.0,0.0,5.0,0.0,2.0,0.0,0.0,3.0,6.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,9.0,0.0,12.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,2.0,397.0,0.0,172.0,10.0,0.0,0.0,0.0,0.0,6.0,3.0,9.0,0.0,0.0,0.0,103.0,0.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,2.0,79.0,5.0,27.0,19.0,1.0,0.0,2.0,1.0,39.0,0.0,187.0,136.0,5.0,0.0,15.0,3.0,0.0,64.0,3.0,0.0,3.0,2.0,7.0,15.0,55.0,0.0,0.0,6.0,0.0,0.0,1.0,1.0,2.0,8.0,5.0,16.0,5.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,12.0,309.0,1.0,71.0,65.0,0.0,0.0,0.0,3.0,19.0,0.0,58.0,0.0,0.0,0.0,40.0,0.0,0.0,6.0,2.0,0.0,0.0,0.0,3.0,26.0,11.0,0.0,0.0,6.0,0.0,1.0,2.0,0.0,0.0,8.0,0.0,27.0,0.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,2.0,423.0,0.0,59.0,23.0,3.0,0.0,0.0,0.0,10.0,4.0,107.0,0.0,1.0,0.0,7.0,1.0,0.0,22.0,0.0,1.0,1.0,18.0,3.0,14.0,17.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,10.0,14.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49993,0.0,206.0,1.0,83.0,40.0,4.0,0.0,1.0,1.0,12.0,0.0,163.0,0.0,0.0,0.0,22.0,4.0,0.0,190.0,2.0,2.0,2.0,4.0,0.0,15.0,4.0,4.0,0.0,0.0,0.0,0.0,9.0,0.0,3.0,3.0,19.0,36.0,1.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49995,14.0,158.0,5.0,66.0,30.0,2.0,0.0,1.0,2.0,7.0,9.0,167.0,60.0,2.0,0.0,129.0,3.0,0.0,33.0,0.0,32.0,4.0,0.0,0.0,4.0,107.0,3.0,0.0,2.0,0.0,0.0,3.0,0.0,4.0,14.0,2.0,2.0,4.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [64]:
train = pd.merge(train_target, info_transac_train,on='client_id')
train = pd.merge(train, counts_train,on='client_id')
train

,client_id,bins,min,max,sum,mean,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,small_group_10,small_group_11,small_group_12,small_group_13,small_group_14,small_group_15,small_group_16,small_group_17,small_group_18,small_group_19,small_group_20,small_group_21,small_group_22,small_group_23,small_group_24,small_group_25,small_group_26,small_group_27,small_group_28,small_group_29,small_group_30,small_group_31,small_group_32,small_group_33,...,small_group_162,small_group_163,small_group_164,small_group_165,small_group_166,small_group_167,small_group_168,small_group_169,small_group_170,small_group_171,small_group_172,small_group_173,small_group_174,small_group_175,small_group_176,small_group_177,small_group_178,small_group_179,small_group_180,small_group_181,small_group_182,small_group_183,small_group_184,small_group_185,small_group_186,small_group_187,small_group_188,small_group_189,small_group_190,small_group_191,small_group_192,small_group_193,small_group_195,small_group_196,small_group_197,small_group_198,small_group_199,small_group_200,small_group_202,small_group_203
0,24662,2,0.074,1227.314,30254.011,34.774725,0.0,174.0,2.0,64.0,33.0,0.0,0.0,0.0,1.0,3.0,0.0,92.0,365.0,0.0,0.0,11.0,0.0,0.0,20.0,0.0,0.0,4.0,3.0,3.0,9.0,16.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1046,0,0.550,1210.506,42548.570,52.015367,1.0,187.0,61.0,47.0,13.0,1.0,0.0,0.0,2.0,8.0,1.0,27.0,3.0,0.0,1.0,79.0,0.0,0.0,142.0,0.0,2.0,0.0,4.0,2.0,5.0,4.0,3.0,0.0,6.0,1.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34089,2,0.043,782.641,26842.816,34.325852,0.0,372.0,0.0,72.0,37.0,10.0,0.0,0.0,0.0,17.0,0.0,47.0,9.0,0.0,0.0,49.0,15.0,1.0,6.0,0.0,2.0,2.0,1.0,5.0,26.0,21.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34848,1,0.043,109.590,15773.126,16.160990,0.0,359.0,1.0,0.0,41.0,0.0,0.0,0.0,0.0,38.0,0.0,116.0,0.0,0.0,0.0,306.0,0.0,0.0,45.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,47076,3,0.432,541.165,12488.375,15.929050,0.0,378.0,0.0,150.0,44.0,0.0,0.0,0.0,0.0,122.0,0.0,33.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,8.0,31.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,14303,1,1.786,9919.347,285001.426,345.456274,0.0,284.0,26.0,31.0,44.0,6.0,2.0,5.0,2.0,24.0,1.0,18.0,51.0,6.0,7.0,1.0,0.0,1.0,21.0,1.0,1.0,12.0,1.0,8.0,9.0,3.0,27.0,0.0,3.0,8.0,0.0,0.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29996,22301,2,0.043,2136.877,34950.321,49.715962,2.0,168.0,9.0,64.0,9.0,0.0,1.0,0.0,1.0,26.0,5.0,96.0,22.0,0.0,0.0,19.0,2.0,0.0,46.0,1.0,2.0,0.0,3.0,1.0,6.0,95.0,20.0,0.0,2.0,0.0,0.0,19.0,5.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299

In [ ]:
info_transac_test = transactions_test.groupby('client_id')['amount_rur'].agg(['min', 'max', 'sum', 'mean'])

amount_transactions_test = transactions_test.groupby(['client_id','small_group'])['amount_rur'].count()
counts_test = amount_transactions_test.reset_index().pivot(index='client_id', columns='small_group',values='amount_rur')
counts_test = counts_test.fillna(0)
counts_test.columns = ['small_group_'+str(i) for i in counts_test.columns]

test = pd.merge(test_target, info_transac_test, on='client_id')
test = pd.merge(test, counts_test, on='client_id')
test

In [78]:
train_columns = train.columns
test_columns = test.columns

In [81]:
for i in train_columns:
    x = False
    for j in test_columns:
        if i == j: x = True
    if x == False: print(i)
print("\n________________________________________\n")
for i in test_columns:
    x = False
    for j in train_columns:
        if i == j: x = True
    if x == False: print(i)

bins
small_group_199
small_group_203

________________________________________

small_group_194
small_group_201


In [82]:
train = train.drop(['small_group_199', 'small_group_203'], axis=1)
test = test.drop(['small_group_194', 'small_group_201'], axis=1)

In [85]:
train

,client_id,bins,min,max,sum,mean,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,small_group_10,small_group_11,small_group_12,small_group_13,small_group_14,small_group_15,small_group_16,small_group_17,small_group_18,small_group_19,small_group_20,small_group_21,small_group_22,small_group_23,small_group_24,small_group_25,small_group_26,small_group_27,small_group_28,small_group_29,small_group_30,small_group_31,small_group_32,small_group_33,...,small_group_160,small_group_161,small_group_162,small_group_163,small_group_164,small_group_165,small_group_166,small_group_167,small_group_168,small_group_169,small_group_170,small_group_171,small_group_172,small_group_173,small_group_174,small_group_175,small_group_176,small_group_177,small_group_178,small_group_179,small_group_180,small_group_181,small_group_182,small_group_183,small_group_184,small_group_185,small_group_186,small_group_187,small_group_188,small_group_189,small_group_190,small_group_191,small_group_192,small_group_193,small_group_195,small_group_196,small_group_197,small_group_198,small_group_200,small_group_202
0,24662,2,0.074,1227.314,30254.011,34.774725,0.0,174.0,2.0,64.0,33.0,0.0,0.0,0.0,1.0,3.0,0.0,92.0,365.0,0.0,0.0,11.0,0.0,0.0,20.0,0.0,0.0,4.0,3.0,3.0,9.0,16.0,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1046,0,0.550,1210.506,42548.570,52.015367,1.0,187.0,61.0,47.0,13.0,1.0,0.0,0.0,2.0,8.0,1.0,27.0,3.0,0.0,1.0,79.0,0.0,0.0,142.0,0.0,2.0,0.0,4.0,2.0,5.0,4.0,3.0,0.0,6.0,1.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34089,2,0.043,782.641,26842.816,34.325852,0.0,372.0,0.0,72.0,37.0,10.0,0.0,0.0,0.0,17.0,0.0,47.0,9.0,0.0,0.0,49.0,15.0,1.0,6.0,0.0,2.0,2.0,1.0,5.0,26.0,21.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,34848,1,0.043,109.590,15773.126,16.160990,0.0,359.0,1.0,0.0,41.0,0.0,0.0,0.0,0.0,38.0,0.0,116.0,0.0,0.0,0.0,306.0,0.0,0.0,45.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,47076,3,0.432,541.165,12488.375,15.929050,0.0,378.0,0.0,150.0,44.0,0.0,0.0,0.0,0.0,122.0,0.0,33.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,2.0,8.0,31.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,14303,1,1.786,9919.347,285001.426,345.456274,0.0,284.0,26.0,31.0,44.0,6.0,2.0,5.0,2.0,24.0,1.0,18.0,51.0,6.0,7.0,1.0,0.0,1.0,21.0,1.0,1.0,12.0,1.0,8.0,9.0,3.0,27.0,0.0,3.0,8.0,0.0,0.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29996,22301,2,0.043,2136.877,34950.321,49.715962,2.0,168.0,9.0,64.0,9.0,0.0,1.0,0.0,1.0,26.0,5.0,96.0,22.0,0.0,0.0,19.0,2.0,0.0,46.0,1.0,2.0,0.0,3.0,1.0,6.0,95.0,20.0,0.0,2.0,0.0,0.0,19.0,5.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299

In [86]:
test

,client_id,min,max,sum,mean,small_group_0,small_group_1,small_group_2,small_group_3,small_group_4,small_group_5,small_group_6,small_group_7,small_group_8,small_group_9,small_group_10,small_group_11,small_group_12,small_group_13,small_group_14,small_group_15,small_group_16,small_group_17,small_group_18,small_group_19,small_group_20,small_group_21,small_group_22,small_group_23,small_group_24,small_group_25,small_group_26,small_group_27,small_group_28,small_group_29,small_group_30,small_group_31,small_group_32,small_group_33,small_group_34,...,small_group_160,small_group_161,small_group_162,small_group_163,small_group_164,small_group_165,small_group_166,small_group_167,small_group_168,small_group_169,small_group_170,small_group_171,small_group_172,small_group_173,small_group_174,small_group_175,small_group_176,small_group_177,small_group_178,small_group_179,small_group_180,small_group_181,small_group_182,small_group_183,small_group_184,small_group_185,small_group_186,small_group_187,small_group_188,small_group_189,small_group_190,small_group_191,small_group_192,small_group_193,small_group_195,small_group_196,small_group_197,small_group_198,small_group_200,small_group_202
0,28571,0.078,306.882,30507.083,42.488974,0.0,278.0,13.0,35.0,35.0,0.0,0.0,0.0,1.0,14.0,3.0,67.0,1.0,0.0,0.0,38.0,28.0,0.0,28.0,0.0,0.0,1.0,0.0,13.0,37.0,15.0,2.0,0.0,2.0,0.0,0.0,4.0,1.0,5.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,27046,0.043,1469.007,39378.016,38.988135,9.0,193.0,68.0,34.0,51.0,0.0,2.0,0.0,1.0,31.0,3.0,231.0,24.0,4.0,0.0,57.0,21.0,0.0,65.0,1.0,0.0,4.0,0.0,3.0,36.0,32.0,4.0,0.0,1.0,0.0,2.0,9.0,2.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13240,1.078,3902.918,50211.127,53.302683,0.0,227.0,3.0,165.0,41.0,0.0,0.0,1.0,2.0,26.0,5.0,226.0,0.0,0.0,0.0,58.0,1.0,0.0,18.0,0.0,1.0,11.0,1.0,3.0,18.0,35.0,0.0,0.0,5.0,0.0,0.0,1.0,2.0,0.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19974,0.432,5865.551,45371.491,53.252924,42.0,305.0,12.0,70.0,49.0,0.0,0.0,0.0,4.0,13.0,1.0,94.0,3.0,2.0,0.0,3.0,5.0,0.0,4.0,1.0,0.0,3.0,1.0,4.0,26.0,32.0,0.0,0.0,2.0,1.0,8.0,2.0,24.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10505,0.043,1921.341,90883.628,81.950972,0.0,516.0,56.0,162.0,47.0,1.0,0.0,1.0,4.0,35.0,2.0,39.0,0.0,0.0,7.0,94.0,2.0,0.0,10.0,1.0,0.0,4.0,4.0,8.0,13.0,0.0,2.0,0.0,6.0,6.0,1.0,3.0,1.0,0.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2565,0.051,3202.483,35397.209,35.397209,0.0,322.0,25.0,38.0,22.0,1.0,0.0,0.0,5.0,17.0,1.0,150.0,78.0,0.0,1.0,71.0,25.0,0.0,6.0,3.0,0.0,7.0,3.0,3.0,33.0,31.0,5.0,0.0,2.0,0.0,7.0,1.0,2.0,0.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,31255,0.129,646.914,17369.096,16.928943,1.0,239.0,1.0,115.0,29.0,5.0,1.0,0.0,2.0,38.0,4.0,213.0,28.0,0.0,0.0,21.0,4.0,0.0,141.0,1.0,13.0,1.0,1.0,4.0,13.0,14.0,0.0,0.0,0.0,0.0,0.0,15.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [91]:
import pandas as pd

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [84]:
X_train = train.drop('bins', axis = 1)
y_train = train['bins']
X_test = test

In [92]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
model_predict = xgb.predict(X_test)

In [94]:
model_predict

array([0, 2, 3, ..., 2, 2, 3])